In [104]:
import numpy as np
import pandas as pd
import akshare as ak
import statsmodels.formula.api as smf

# ==========================================
# 1. 基金数据准备
# ==========================================
fund_code = "166001"
start_date = "20190101"
fund_individual_basic_info_xq_df = ak.fund_individual_basic_info_xq(symbol=fund_code)
fund_net_value_df = ak.fund_open_fund_info_em(
    symbol=fund_code,
    indicator="累计净值走势",
    )
fund_net_value_df["净值日期"] = pd.to_datetime(fund_net_value_df["净值日期"])
fund_df = fund_net_value_df[fund_net_value_df["净值日期"] >= start_date]
fund_df = fund_df.set_index("净值日期", drop=True)
fund_df['日度回报率'] = fund_df['累计净值'].pct_change()
fund_df.dropna(subset=['日度回报率'], inplace=True)
# ==========================================
# 2. 因子数据准备，读取参考文件
# ==========================================
file_path = '/Users/sakurason/Library/Mobile Documents/com~apple~CloudDocs/Documents/文件/My Study/大三上/2025秋 人工智能与投资分析/Case/Our Case/参考因子数据/CAPM单因子模型（经典算法）日收益率（截至到20250630）.csv'
factor_capm = pd.read_csv(file_path)
factor_capm['date'] = pd.to_datetime(factor_capm['date'])
factor_capm.set_index('date', inplace=True)
# ==========================================
# 3. 数据合并与预处理
# ==========================================
capm_data = pd.merge(fund_df[['日度回报率']], factor_capm, left_index=True, right_index=True, how='inner')
capm_data['Ri_Rf'] = capm_data['日度回报率'] - capm_data['RF']
capm_data['Market'] = capm_data['MKT'] - capm_data['RF']
# ==========================================
# 4. CAPM回归分析
# ==========================================
formula = 'Ri_Rf ~ Market'
model = smf.ols(formula=formula, data=capm_data).fit()
alpha_daily = model.params['Intercept']
alpha_annualized = alpha_daily * 250
print(model.summary())
print(f"基金 {fund_code} 年化超额收益为{alpha_annualized:.2%}")


                            OLS Regression Results                            
Dep. Variable:                  Ri_Rf   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     5035.
Date:                Tue, 18 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:43:21   Log-Likelihood:                 6818.3
No. Observations:                1572   AIC:                        -1.363e+04
Df Residuals:                    1570   BIC:                        -1.362e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   6.697e-05   7.99e-05      0.839      0.4

In [110]:
fund_df

,累计净值,日度回报率
净值日期,,
2019-01-03,1.6812,-0.005560
2019-01-04,1.6976,0.009755
2019-01-07,1.7100,0.007304
2019-01-08,1.7108,0.000468
2019-01-09,1.7131,0.001344
...,...,...
2025-11-11,3.1539,-0.004168
2025-11-12,3.1558,0.000602
2025-11-13,3.2035,0.015115


In [105]:
# ==========================================
# 5. 择时能力分析
# ==========================================
capm_data['Market_sq'] = capm_data['Ri_Rf'] ** 2
formula = 'Ri_Rf ~ Market + Market_sq'
model = smf.ols(formula=formula, data=capm_data).fit()
alpha_daily = model.params['Intercept']
alpha_annualized = alpha_daily * 250
print(model.summary())
print(f"基金 {fund_code} 年化超额收益为{alpha_annualized:.2%}")

                            OLS Regression Results                            
Dep. Variable:                  Ri_Rf   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.763
Method:                 Least Squares   F-statistic:                     2528.
Date:                Tue, 18 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:44:02   Log-Likelihood:                 6821.2
No. Observations:                1572   AIC:                        -1.364e+04
Df Residuals:                    1569   BIC:                        -1.362e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0001   8.65e-05      1.698      0.0

In [106]:
# ==========================================
# 1. # Carhart四因子数据准备，读取参考文件
# ==========================================
file_path = '/Users/sakurason/Library/Mobile Documents/com~apple~CloudDocs/Documents/文件/My Study/大三上/2025秋 人工智能与投资分析/Case/Our Case/参考因子数据/Carhart-四因子模型（经典算法）日收益率（截至到20250630）.csv'
factor_car = pd.read_csv(file_path)
factor_car['date'] = pd.to_datetime(factor_car['date'])
factor_car.set_index('date', inplace=True)
# ==========================================
# 2. 数据合并与预处理
# ==========================================
car_data = pd.merge(fund_df[['日度回报率']], factor_car, left_index=True, right_index=True, how='inner')
car_data['Ri_Rf'] = car_data['日度回报率'] - car_data['RF']
car_data['Market'] = car_data['MKT'] - car_data['RF']
# ==========================================
# 3. Carhart四因子回归分析
# ==========================================
formula = 'Ri_Rf ~ Market + SMB + HML + UMD'
model = smf.ols(formula=formula, data=car_data).fit()
alpha_daily = model.params['Intercept']
alpha_annualized = alpha_daily * 250
p_value = model.pvalues['Intercept']
print(model.summary())
print(f"基金 {fund_code} 年化超额收益为{alpha_annualized:.2%}")


                            OLS Regression Results                            
Dep. Variable:                  Ri_Rf   R-squared:                       0.824
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     1838.
Date:                Tue, 18 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:44:21   Log-Likelihood:                 7056.1
No. Observations:                1572   AIC:                        -1.410e+04
Df Residuals:                    1567   BIC:                        -1.408e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   9.189e-05   6.87e-05      1.337      0.1

In [ ]:
# ==========================================
# 1. # 模型因子拓展：读取DHS因子数据
# ==========================================
file_path = '/Users/sakurason/Library/Mobile Documents/com~apple~CloudDocs/Documents/文件/My Study/大三上/2025秋 人工智能与投资分析/Case/Our Case/参考因子数据/Daniel-Hirshleifer-Sun-三因子模型（经典算法）日收益率（截至到20250630）.csv'
factor_dhs = pd.read_csv(file_path)
factor_dhs['date'] = pd.to_datetime(factor_dhs['date'])
factor_dhs.set_index('date', inplace=True)
# ==========================================
# 2. 数据合并
# ==========================================
full_data=pd.merge(car_data,factor_dhs[['R_fin','R_pead']],left_index=True,right_index=True)
full_data.dropna(inplace=True)
# ==========================================
# 3. 运行合并“六因子”回归分析
# ==========================================
formula = 'Ri_Rf ~ Market + SMB + HML + UMD + R_pead + R_fin'
model = smf.ols(formula=formula, data=full_data).fit()
alpha_daily = model.params['Intercept']
alpha_annualized = alpha_daily * 250
print(model.summary())
print(f"基金 {fund_code} 年化超额收益为{alpha_annualized:.2%}")
# ==========================================
# 4. 多重共线性检查：VIF
# ==========================================
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
X_factors = full_data[['Market', 'SMB', 'HML', 'UMD', 'R_fin', 'R_pead']]
def get_vif_summary(data, formula):
    vif_data = pd.DataFrame()
    vif_data["Factor"] = X_factors.columns
    vif_data["VIF"] = [variance_inflation_factor(X_factors.values, i) 
                       for i in range(X_factors.shape[1])]
    vif_data = vif_data.sort_values("VIF", ascending=False).reset_index(drop=True)
    return vif_data
vif_summary = get_vif_summary(full_data, "Market + SMB + HML + UMD + R_fin + R_pead ~ RET_1D")
print(vif_summary)

                            OLS Regression Results                            
Dep. Variable:                  Ri_Rf   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.825
Method:                 Least Squares   F-statistic:                     1233.
Date:                Tue, 18 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:44:38   Log-Likelihood:                 7061.1
No. Observations:                1572   AIC:                        -1.411e+04
Df Residuals:                    1565   BIC:                        -1.407e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   7.766e-05   6.89e-05      1.127      0.2

In [109]:
# ==========================================
# 5. 择时能力分析
# ==========================================
car_data['Market_sq'] = car_data['Ri_Rf'] ** 2
formula = 'Ri_Rf ~ Market + Market_sq + SMB + HML + UMD'
model =smf.ols(formula, car_data).fit()
alpha_daily = model.params['Intercept']
alpha_annualized = alpha_daily * 250
print(model.summary())
print(f"基金 {fund_code} 年化超额收益为{alpha_annualized:.2%}")

                            OLS Regression Results                            
Dep. Variable:                  Ri_Rf   R-squared:                       0.825
Model:                            OLS   Adj. R-squared:                  0.825
Method:                 Least Squares   F-statistic:                     1481.
Date:                Tue, 18 Nov 2025   Prob (F-statistic):               0.00
Time:                        17:47:12   Log-Likelihood:                 7060.9
No. Observations:                1572   AIC:                        -1.411e+04
Df Residuals:                    1566   BIC:                        -1.408e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0002   7.44e-05      2.448      0.0